In [1]:
import torch, torchvision
import sys # Python system library needed to load custom functions
import math # module with access to mathematical functions
import os # for changing the directory

import numpy as np  # for performing calculations on numerical arrays
import pandas as pd  # home of the DataFrame construct, _the_ most important object for Data Science
import librosa
import matplotlib.pyplot as plt  # allows creation of insightful plots

sys.path.append('../audio_preprocessing')
sys.path.append('../src')
sys.path.append('../model_training_utils')


import preprocessing_func_2
from generator_to_dataset_2 import NormalisedDataSet
from gdsc_utils import PROJECT_DIR
import model_training
import model_eval

os.chdir(PROJECT_DIR) # changing our directory to root

In [2]:
df = pd.read_csv('data/metadata.csv')

In [3]:
def process_and_save_wav(paths, labels, chunk_size:int=66150, wav_max_amplitude:float=0.5, sr=44100):
    new_paths = []
    new_labels = []
    i = 0
    for (path, label) in zip(paths, labels):
        wav = preprocessing_func_2.load_wav(path=path)
        wav = preprocessing_func_2.normalise_wav(wav=wav, wav_max_amplitude=wav_max_amplitude)
        peaks = preprocessing_func_2.find_wav_peaks(wav=wav, distance_between_peaks=chunk_size)
        if peaks is None:
            wav = preprocessing_func_2.process_small_wav(wav=wav, chunk_size=chunk_size)
            if len(wav) != 66150:
                print("process_small_wav gone wrong")
                print(f"length of wav is {len(wav)}")
                print(path, label)
                print()
                new_paths.append(path)
                new_labels.append(label)
            continue
        for chunks in preprocessing_func_2.split_wav_by_peaks(wav=wav, peaks=peaks, chunk_size=chunk_size):
            if len(chunks) != 66150:
                print("split_wav_by_peaks gone wrong")
                print(f"length of wav is {len(chunks)}")
                print(path, label)
                print()
                new_paths.append(path)
                new_labels.append(label)
    return new_paths, new_labels

In [4]:
paths, labels = list(df["path"]), list(df["label"])

new_paths, new_labels = process_and_save_wav(paths, labels)